<a href="https://colab.research.google.com/github/ash7-g/AI-1BM23CS400/blob/main/Week9_Resolution_1BM23CS400.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
def is_variable(term):
    """Check if a term is a variable (lowercase without parentheses)."""
    return term.islower() and "(" not in term and ")" not in term

def split_literal(literal):
    """Helper function to handle splitting of literal into predicate and arguments."""
    if "(" in literal:
        pred, args = literal.split("(")
        args = args[:-1]  # Remove closing ')'
        args = args.split(",")
        return pred, args
    else:
        return literal, []

def unify_literal(lit1, lit2):
    """Unify two literals if possible, returning a substitution map or None."""
    # If both literals are identical, no unification is needed
    if lit1 == lit2:
        return {}

    # Handle negation: If one literal is negated, remove the negation for comparison
    if lit1.startswith("¬"):
        lit1 = lit1[1:]
    if lit2.startswith("¬"):
        lit2 = lit2[1:]

    # Now split the literals into predicate and arguments
    pred1, args1 = split_literal(lit1)
    pred2, args2 = split_literal(lit2)

    if pred1 != pred2:
        return None

    subst = {}
    for a1, a2 in zip(args1, args2):
        if a1 != a2:
            if is_variable(a1):
                subst[a1] = a2
            elif is_variable(a2):
                subst[a2] = a1
            else:
                return None
    return subst

def apply_substitution(clause, substitution):
    """Apply a substitution to a clause."""
    new_literals = []
    for lit in clause.literals:
        new_lit = lit
        for var, const in substitution.items():
            new_lit = new_lit.replace(var, const)
        new_literals.append(new_lit)
    return Clause(new_literals)

class Clause:
    def __init__(self, literals):
        self.literals = set(literals)

    def __repr__(self):
        return " ∨ ".join(sorted(self.literals))

    def resolve(self, other):
        """Perform resolution between two clauses."""
        resolvents = []
        for literal in self.literals:
            for other_literal in other.literals:
                if literal.startswith("¬"):
                    neg_literal = literal[1:]
                else:
                    neg_literal = "¬" + literal

                unifier = unify_literal(literal, neg_literal)
                if unifier is not None:
                    new_clause = Clause((self.literals - {literal}) | (other.literals - {other_literal}))
                    resolvents.append(apply_substitution(new_clause, unifier))
        return resolvents

def PL_RESOLUTION(KB, query):
    """Performs First-Order Logic Resolution."""
    # Negate the query and add it to the knowledge base
    negated_query = Clause([f"¬{query}"])
    clauses = KB + [negated_query]

    new = set()
    seen_pairs = set()

    while True:
        pairs = [(clauses[i], clauses[j]) for i in range(len(clauses)) for j in range(i + 1, len(clauses))]

        for ci, cj in pairs:
            if (ci, cj) in seen_pairs or (cj, ci) in seen_pairs:
                continue
            seen_pairs.add((ci, cj))

            resolvents = ci.resolve(cj)
            for resolvent in resolvents:
                if not resolvent.literals:
                    return True  # Found the empty clause, meaning a contradiction was derived
                new.add(frozenset(resolvent.literals))

        # If no new clauses were generated, return False (no contradiction can be derived)
        if new.issubset(set(map(frozenset, (c.literals for c in clauses)))):  # Check if new clauses are subsets
            return False

        # Add the new clauses to the set of clauses
        for literals in new - set(map(frozenset, (c.literals for c in clauses))):
            clauses.append(Clause(list(literals)))

        new.clear()

# Example knowledge base in First-Order Logic
KB = [
    Clause(["Mother(Leela, Oshin)"]),
    Clause(["Alive(Leela)"]),
    Clause(["¬Mother(x, y) ∨ Parent(x, y)"]),
    Clause(["¬Parent(x, y) ∨ ¬Alive(x) ∨ Older(x, y)"]),
    Clause(["¬Mother(x, y) ∨ Older(x, y)"])  # Converted rule: Mother(x, y) → Older(x, y)
]

# Query: Prove Older(Leela, Oshin)
query = "Older(Leela, Oshin)"
if PL_RESOLUTION(KB, query):
    print(f"The conclusion '{query}' is proven by resolution.")
else:
    print(f"The conclusion '{query}' cannot be proven.")



The conclusion 'Older(Leela, Oshin)' is proven by resolution.
